# Наивный Байесовский классификатор 

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('spam.csv', encoding = 'latin-1', 
                 usecols=['v1', 'v2'])
df.rename(columns={'v1': 'target', 'v2': 'message'}, inplace=True)
df.target.replace({'spam': 1, 'ham': 0}, inplace=True)
df.head()

,target,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
def preprocess_message(m):
    # split into tokens
    words = word_tokenize(m)
    
    # filter out words with length <= 2
    words = [w for w in words if len(w) > 2]
    
    # remove stop-words
    sw = stopwords.words('english')
    words = [word for word in words if word not in sw]
    
    # stem
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

In [5]:
df.message = df.message.apply(preprocess_message)
df.head()

,target,message
0,0,jurong point crazy.. Avail bugi great world bu...
1,0,lar ... Joke wif oni ...
2,1,Free entri wkli comp win Cup final tkt 21st Ma...
3,0,dun say earli hor ... alreadi say ...
4,0,Nah n't think goe usf live around though


In [6]:
df.target.value_counts()

0    4825
1     747
Name: target, dtype: int64

### Формула которую нам придется запрограммировать

$$ 
P(spam | message ) = \frac{P(message | spam)  P(spam)}{P(message)}
$$

Чтобы получить ответ, нужно перемножить/разделить 3 величины. Чтобы их посчитать, придется расписать формулу чуть поподробнее:

$$ 
\frac{P(message | spam)  P(spam)}{P(message)} = \frac{P(word_1 \cap word_2 \cap word_3 ... \cap word_n | spam) P(spam)}{P(word_1 \cap word_2 \cap ... \cap word_n)}
$$

Считать совместную вероятность вхождения слов в сообщение - это сложно. Вот тут-то и появляется "наивность". Заключается она в том что мы, для упрощения вычислений, сделаем грубое предположение: появление разных слов в предложении это события независимые. 

В таком случае вероятность одновременного появления слов превратится в перемножение вероятностей встретить каждое слово по-отдельности.

$$ 
P(word_1 \cap word_2 \cap word_3 ... \cap word_n | spam) \approx P(word_1|spam)P(word_2|spam) ... P(word_n|spam) 
$$

Вот и весь смысл наивного Байесовского классификатора. Теперь осталось посчитать все формулы по-отдельности, а затем скомбинировать.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(df)
X_train.shape, X_valid.shape

((4179, 2), (1393, 2))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
class SpamDetector():
    
    def __init__(self):
        pass
    
    # typical machine learning model interface
    def fit(self, dataframe):
        self.__checkdf(dataframe)
        pass # do smth useful
        self.p_spam = self.__p_spam(dataframe)
        
    
    def predict(self, dataframe):
        self.__checkdf(dataframe)
        pass # do smth useful
        return np.array([1 if self.p_spam >= 0.5 else 0] * dataframe.shape[0])
    
    def predict_proba(self, dataframe):
        self.__checkdf(dataframe)
        pass # do smth useful
        return np.array([[1.0 - self.p_spam, self.p_spam]] * dataframe.shape[0])
       
    
    # helper functions
    def __checkdf(self, dataframe):
        assert all([c in ['target', 'message'] for c in dataframe.columns])
        
    def __p_spam(self, dataframe):
        return dataframe.target.mean()

    def __p_message_if_spam(self, dataFrame):
        countvec = CountVectorizer()
        countvec.fit(dataFrame[dataframe.target == 1].message)
        for i in countvec.vocabulary_:
            countvec.vocabulary_
        pass

    def __p_message():
        pass # do we really need this? try to guess ;)

In [10]:
sfilter = SpamDetector()
sfilter.fit(X_train)

In [11]:
y_hat_p = sfilter.predict_proba(X_valid)
y_hat = sfilter.predict(X_valid)
y_hat_p

array([[ 0.86958603,  0.13041397],
       [ 0.86958603,  0.13041397],
       [ 0.86958603,  0.13041397],
       ..., 
       [ 0.86958603,  0.13041397],
       [ 0.86958603,  0.13041397],
       [ 0.86958603,  0.13041397]])

In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_hat, X_valid.target))

             precision    recall  f1-score   support

          0       1.00      0.85      0.92      1393
          1       0.00      0.00      0.00         0

avg / total       1.00      0.85      0.92      1393



C:\Users\MASHA\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
